# ADM - Homework 05

## Visit the Wikipedia hyperlinks graph!

#### Group 28 - Hafiz Muhammad, Marco Minici, François Chassaing 

Importing the custom functions coded to solve the problem.

In [1]:
from hw05_FUNCTIONS import *

Importing all the necessary python packages to perform the analysis requested.

In [2]:
import pandas as pd
import numpy as np
import networkx as nx
import collections
from os.path import isfile
from statistics import median

### RQ1

**Build the graph G=(V, E), where V is the set of articles and E the hyperlinks among them, and provide its basic information:** 

**1. If it is direct or not?**

To answer this question we have to add edges and nodes to the networkx librarary. There is function called **nx.is_directed(G)** whether the graph is directed or not. 

In [24]:
G = nx.DiGraph() # notice directed graph
f = open("data/wiki-topcats-page-names.txt", "r")
for row in f:
    lrow = row.split()
    name = ' '.join(lrow[1:])
    G.add_node(int(lrow[0]))
    G.node[int(lrow[0])]["name"] = name
f.close()

After adding nodes we have to add edges for these lets use **wiki-topcats-reduced.txt** file so that we have insights about the graph. 

In [25]:
flink = open("data/wiki-topcats-reduced.txt", "r")
for i,row in enumerate(flink):
    lrow = row.split()
    G.add_edge(int(lrow[0]), int(lrow[1])) #adding edges
flink.close()

In [26]:
nx.is_directed(G) # this function will tell us whether it is directed or not. 

True

*This tells us that graph we are trying to work with is directed because edge meaning <node1,node2> in node1 wikipedia article got a link to the node2 wikipedia article.* so this answer the question whether the graph is directed or not.* **The graph is directed**

### The number of nodes? The number of edges? 
for this we are going to use simple functions like **number_of_nodes** and **number_of_edges()**. 

In [27]:
G.number_of_nodes()

1791489

In [28]:
G.number_of_edges()

2645247

The number of nodes are **1791489** and number of edges are **2174451**. 

### The average node degree. Is the graph dense?

To answer this we will use *density()** and degree function. 

In [30]:
degreeM = 0
for node in G.node:
    degreeM += G.degree(node)
print(degreeM/G.number_of_nodes())

2.9531267007500466


In [31]:
nx.density(G)

8.242105726496763e-07

So to answer above question the average node degree is *2.4275348606661833* and density is *1.3550383037263901e-06*. which says that the graph is not much dense.

Lets move to reasearch question 2. 

## RQ2:
Given a category C_0 = \{article_1, article_2, \dots \} as input we want to rank all of the nodes in V according to the given criteria discussed in given question. 


Declaring some preliminary variables that will be used multiple times sequently in the code.

In [3]:
#this is the folder path where all the data are stored
dataFolder = "./data/"
#base filename for all wiki data
basefilename = "wiki-topcats-"
#file extension of wiki data
ext = ".txt"
#name of all wiki data files
CATEGORIES = "categories"
REDUCED_GRAPH = "reduced"
PAGE_NAMES = "page-names"

Reading the file containing the *categories*.

This file contains a list of categories. Each category is a wikipedia articles collection, where each article belongs to the specific category.

The operation performed are:

    - File reading with pandas package.
    - Cleaning the "category" column deleting the redundant "Category:" string, let the value be only the category name.
    - Retain only the categories which have more than 3500 articles inside
   

In [4]:
#let's open the wiki_categories.txt file and delete all categories
#with less than 3500 articles
#
#read the category file
categories = pd.read_csv(dataFolder+basefilename+CATEGORIES+ext, sep = ";", header = None , names = ["Category","List_of_articles"])
#clean the category column
#Before: "Category: category_name"
#After: "category_name"
categories["Category"] = categories["Category"].apply(lambda x:x.split(":")[1])
#defining a function that delete all categories with less than 3500 articles
ARTICLES_THRESHOLD = 3500
#Scanning each value of list_of_articles, if the number of articles goes beyond the threshold then retain it
#otherwise return an empty string
categories["List_of_articles"] = categories["List_of_articles"].apply(lambda x: x if len(x.strip().split(" ")) >= ARTICLES_THRESHOLD else "")
#delete all rows with empty string as "list_of_articles"
categories = categories[categories["List_of_articles"] != ""]

Now the graph representing the wikipedia articles network must be built, before creating the mentioned data structure it is needed to check if the nodes contained in the *category* file and the nodes in the *graph* are the same. This is a necessary condition to perform a consistent analysis.

The consistency check is performed as follows:

    - Preliminary reading all the edges of the graph using the pandas library.
    - Create the set of nodes which compare at least in one of the edges, either as the starting node or as the final node.
    - Create the set of nodes which compare at least in one of the categories, an integer transformation from string is necessary, this allows to make a comparison with the nodes of the graph.
    - Perform the intersection between the set of nodes found in the graph and the set of nodes found in the categories file. 
    
The final set produced will be sequently used to normalize the categories and the graph file, retaining only the nodes present in both.
    

In [5]:
#Now it is needed to build the final graph, but it must be checked that the nodes into the reduced graph and 
#the nodes into the categories files are the same.
#The set of the nodes into the categories must be built.
#The set of nodes into the reduced graph must be built.
#the intersection between these two sets must be computed.
#only the edges involving nodes of the intersection set must be added to the final graph

#let's open the reduced graph file and create the set of nodes 
reduced_graph = pd.read_csv(dataFolder+basefilename+REDUCED_GRAPH+ext, sep = "\t", header = None, names = ["Node_1","Node_2"])
#create the set of the first column
set_node_1 = set(reduced_graph["Node_1"].values.tolist())
#create the set of the second column
set_node_2 = set(reduced_graph["Node_2"].values.tolist())
#create the set of the nodes into the reduced graph through the union operation of the two previous created sets
set_reduced_graph_nodes = set.union(set_node_1,set_node_2)
#print the size for debugging
print("size of set_node_1:= "+str(len(set_node_1)))
print("size of set_node_2:= "+str(len(set_node_2)))
print("size of set_reduced_graph_nodes:= "+str(len(set_reduced_graph_nodes)))
#The two previous sets are not useful anymore therefore they are deleted from the main memory
del set_node_1
del set_node_2

#Let's create the set of nodes into the categories
#create initially an empty set
set_categories_nodes = set()
#in order to perform the intersection function the nodes must be represented in the same format
#since the nodes into the @reduced_graph dataframe are integer then the "int" type is chosen
categories["List_of_articles"].apply(lambda x: set_categories_nodes.update(set(map(int, x.strip().split(" ")))))

#compute the final set of nodes
set_of_nodes = set.intersection(set_categories_nodes, set_reduced_graph_nodes)
#print the size for debugging
print("size of set_categories_nodes:= "+str(len(set_categories_nodes)))
print("size of set_reduced_graph_nodes:= "+str(len(set_reduced_graph_nodes)))
print("size of set_of_nodes:= "+str(len(set_of_nodes)))
#once the final set is computed the other sets can be deleted to free the memory
del set_categories_nodes
del set_reduced_graph_nodes

size of set_node_1:= 428957
size of set_node_2:= 352518
size of set_reduced_graph_nodes:= 461193
size of set_categories_nodes:= 546237
size of set_reduced_graph_nodes:= 461193
size of set_of_nodes:= 461193


Instatiate the graph representing the wiki articles network.

It has been chosen a *networx.DiGraph* class because a general edge $<Node_1, Node_2>$ represents [the presence into the $Node_1$ wiki article of a link which points to the wiki article $Node_2$.](https://snap.stanford.edu/data/wiki-topcats.html)

This kind of relationship must be encoded as a directed graph.

Before adding the edge a filtering is applied, checking if both nodes of the edges are present in the set of nodes previously computed.

In [6]:
#now it is possible to read line by line all the edges of the reduced graph
#and adding it to the final graph only if they belong to the @set_of_nodes computed
final_graph = nx.DiGraph()

#build the graph
reduced_graph.apply(lambda edge: final_graph.add_edge(edge[0],edge[1]) if filterEdges(edge,set_of_nodes) else "" ,axis = 1)

0          None
1          None
2          None
3          None
4          None
5          None
6          None
7          None
8          None
9          None
10         None
11         None
12         None
13         None
14         None
15         None
16         None
17         None
18         None
19         None
20         None
21         None
22         None
23         None
24         None
25         None
26         None
27         None
28         None
29         None
           ... 
2645217    None
2645218    None
2645219    None
2645220    None
2645221    None
2645222    None
2645223    None
2645224    None
2645225    None
2645226    None
2645227    None
2645228    None
2645229    None
2645230    None
2645231    None
2645232    None
2645233    None
2645234    None
2645235    None
2645236    None
2645237    None
2645238    None
2645239    None
2645240    None
2645241    None
2645242    None
2645243    None
2645244    None
2645245    None
2645246    None
Length: 2645247, dtype: 

In [14]:
#CHUNK INTO WHICH THE AVERAGE NODE DEGREE AND THE DENSITY OF THE GRAPH MUST BE COMPUTED

The same kind of filtering operation is performed on the categories file deleting all the nodes not belonging to the intersection set previously computed.

In [7]:
#the category dataframe must involve only "good nodes" too
categories["List_of_articles"] = categories["List_of_articles"].apply(lambda x: filter_nodes_in_categories(x,set_of_nodes))

To speed-up the analysis, the distance between each category is computed offline and stored in a file(in this case, it is named *category_distances.npy*)representing a (C_n,C_n)-shaped matrix where C_n is equal to the number of categories contained in the dataset.

If the file isn't found on the filesystem, the matrix is computed in real-time.

In [8]:
category_distances = retrieveCategoryDistances(filename = "category_distances.npy", final_graph = final_graph ,categories=categories)

To simplify and to speed-up the analysis, instead to have the categories as a *pandas.DataFrame* it is preferable to code them as a *python built-in dictionary*, where the keys are the category name and the values are the list of wikipedia articles belonging to that category.

In [9]:
#compute a dictionary that has as keys the categories and as values the list of articles
#that belongs to that category
category_dictionary = {}
#for each category
for i in range(categories.shape[0]):
    #assign to that category the list of articles
    category_dictionary[i] = list(map(int, categories.iloc[i]['List_of_articles'].split(" ")))

### Example of ranking computation

The first step is, given an input category( in this example, it is $C_0$ ), to compute the block ranking vector where the blocks are represented by the categories.

The first category of the rank, $C_0$, always corresponds to the input category. The order of the remaining categories is given by:

$$distance(C_0,C_i) = median(ShortestPath(C_0, C_i))$$

The lower is the distance from $C_0$, the higher is the $C_i$ position in the rank. 

#### Shortest path computation procedure explained:

Given an input category $C_0$ and another category $C_i$, it exists at most one shortest path that goes from the category $C_0$ to each of the nodes $C_i$.

Therefore, only for the purpose of computing the block ranking, it is added a *fake node* which has an in-degree equal to 0 and has as many out-edges as nodes contained in the $C_0$ category.

The image that follows will explain better the mentioned situation.

The *green* node represents the fake node, the *yellow* ones belongs to the input category and the *red* represents the rest of the graph. 

<img src="graph.png">

Once the *fake node* and the related edges are added, the distance computation can be performed.

It is known that the graph has the following properties:

    - It is directed.
    - It isn't weighted.
    
Given these two properties to compute the shortest paths from the *fake_node* to all the nodes in the graph, it is preferrable to use the [Breadth-First-Search](https://en.wikipedia.org/wiki/Breadth-first_search) algorithm. 
The neighbours of the *root* node are marked as 1-distanced, each time a new *level* is explored the counter variable is increased, so the neighbours of the neighbours of the *root* node will be marked as 2-distanced and so on and so forth.

The BFS will be performed only one time on the *fake_node* and so the time-complexity of the algorithm to compute the distance between one input category and all the others in the world is equal to the BFS time-complexity which it is $O(|V| + |E|)$.

At the end of the procedure, the *fake node* and all the incident edges are deleted.

In [10]:
#declare the input category for the example
input_category = 0

In [11]:
#compute the block ranking vector
blockRanking = computeBlockRanking(input_category,category_distances)

The final node ranking algorithm follows all the steps explained in the [assignment](https://github.com/CriMenghini/ADM-2018/tree/master/Homework_5)

In [12]:
#finally compute the nodes ranking
compute_nodes_ranking(allGraph = final_graph, block_ranking = blockRanking, categories = category_dictionary)

The top-10 ranked articles are the following: 
81941 82322 82082 82089 82091 81871 81878 82346 82084 81267
The entire ranking is saved on the filesystem
